In [157]:

# Data to insert
market_json = [
    {
        "layout": "1BDRM",
        "pf_rent": 2000
    },
    {
        "layout": "2BDRM",
        "pf_rent": 3000
    }
]

rental_assumptions_json = [
    {
        "id": 1,
        "rent_type": "fixed",
        "vacate_flag": 1,
        "layout": "1BDRM",
        "square_feet": 1500,
        "vacate_month": 3,
        "current_rent": 1500
    },
    {
        "id": 2,
        "rent_type": "fixed",
        "vacate_flag": 1,
        "layout": "1BDRM",
        "square_feet": 1500,
        "vacate_month": 3,
        "current_rent": 1500
    },
    {
        "id": 3,
        "rent_type": "fixed",
        "vacate_flag": 0,
        "layout": "1BDRM",
        "square_feet": 1500,
        "vacate_month": 3,
        "current_rent": 1600
    }
]


In [158]:
import gspread
from google.oauth2.service_account import Credentials

# === Setup Google Sheets credentials ===
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

creds = Credentials.from_service_account_file(
    "../uw_backend/secrets/service_account_key.json",
    scopes=scopes
)
client = gspread.authorize(creds)

# === Sheet References ===
spreadsheet = client.open_by_key("1Q6WwG0FxSLXDD9U4BYit_rxUTUdEkEePkpsejt4ux20")
market_ws = spreadsheet.worksheet("Market Rent Assumptions")

# === Row Tracking ===
market_start_row = 2
market_num_rows = len(market_json)
market_end_row = market_start_row + market_num_rows - 1

# These define which rows the formulas will reference in Rental Assumptions
rental_start_row = 2
num_rental_rows = len(rental_assumptions_json)
rental_end_row = rental_start_row + num_rental_rows - 1

# Now update the range references for the formula
rental_d_range = f"'Rental Assumptions'!$D${rental_start_row}:$D${rental_end_row}"
rental_h_range = f"'Rental Assumptions'!$H${rental_start_row}:$H${rental_end_row}"
print(f"rental_d_range: {rental_d_range}")
print(f"rental_h_range: {rental_h_range}")
# === Insert blank rows in Market Rent Assumptions ===
market_ws.insert_rows([[]] * market_num_rows, row=market_start_row)

# === Prepare data to insert ===
rows_to_add = []
for i, entry in enumerate(market_json):
    current_row = market_start_row + i  # the row where this entry will go
    averageif_formula = (
        f"=IFERROR(AVERAGEIF({rental_d_range},A{current_row},{rental_h_range}),0)"
    )
    row = [
        entry["layout"],   # Column A
        "",                # Column B
        entry["pf_rent"],  # Column C
        averageif_formula  # Column D
    ]
    rows_to_add.append(row)

# === Push values into sheet using USER_ENTERED mode ===
range_str = f"'Market Rent Assumptions'!A{market_start_row}:D{market_end_row}"
spreadsheet.values_update(
    range_str,
    params={"valueInputOption": "USER_ENTERED"},
    body={"values": rows_to_add}
)

print(f"✅ Inserted {market_num_rows} rows into 'Market Rent Assumptions' starting at row {market_start_row}.")

rental_d_range: 'Rental Assumptions'!$D$2:$D$4
rental_h_range: 'Rental Assumptions'!$H$2:$H$4
✅ Inserted 2 rows into 'Market Rent Assumptions' starting at row 2.


In [159]:
# Target sheet
rental_ws = spreadsheet.worksheet("Rental Assumptions")

# Define where to insert in Rental Assumptions
rental_start_row = 2
rental_num_rows = len(rental_assumptions_json)
rental_end_row = rental_start_row + rental_num_rows - 1
total_row_index = rental_end_row + 1

# === Construct rows with formulas ===
rental_rows = []
for i, item in enumerate(rental_assumptions_json):
    row_num = rental_start_row + i

    pf_rent_formula = (
        f"=IFERROR("
        f"IF(OR(C{row_num}=1,C{row_num}=2),"
        f"INDEX('Market Rent Assumptions'!$C${market_start_row}:$C${market_end_row}, "
        f"MATCH(D{row_num}, 'Market Rent Assumptions'!$A${market_start_row}:$A${market_end_row}, 0)),"
        f"H{row_num}),"
        f"H{row_num})"
    )

    annual_pf_rent_formula = f"=I{row_num}*12"

    rental_rows.append([
        i + 1,                                        # A: Units
        item["rent_type"],                           # B: Rent Type
        item["vacate_flag"],                         # C: Vacate Flag
        item["layout"],                              # D: Layout
        item["square_feet"],                         # E: Sq. Ft.
        item["vacate_month"],                        # F: Vacated Month
        f"=EOMONTH(Assumptions!$F$2,F{row_num})",    # G: Vacated
        item["current_rent"],                        # H: Current Rent
        pf_rent_formula,                             # I: Pro Forma Rent
        annual_pf_rent_formula                       # J: Annual PF Rent
    ])

# === Step 1: Insert blank rows with formatting ===
spreadsheet.batch_update({
    "requests": [
        {
            "insertDimension": {
                "range": {
                    "sheetId": rental_ws._properties["sheetId"],
                    "dimension": "ROWS",
                    "startIndex": rental_start_row - 1,
                    "endIndex": rental_start_row - 1 + rental_num_rows
                },
                "inheritFromBefore": True
            }
        },
        {
            "repeatCell": {
                "range": {
                    "sheetId": rental_ws._properties["sheetId"],
                    "startRowIndex": rental_start_row - 1,
                    "endRowIndex": rental_start_row - 1 + rental_num_rows,
                    "startColumnIndex": 6,
                    "endColumnIndex": 7
                },
                "cell": {
                    "userEnteredFormat": {
                        "numberFormat": {
                            "type": "DATE",
                            "pattern": "mm/dd/yyyy"
                        }
                    }
                },
                "fields": "userEnteredFormat.numberFormat"
            }
        }
    ]
})

# === Step 2: Write rows to Rental Assumptions ===
spreadsheet.values_update(
    f"'Rental Assumptions'!A{rental_start_row}:J{rental_end_row}",
    params={"valueInputOption": "USER_ENTERED"},
    body={"values": rental_rows}
)

print(f"✅ Inserted and formatted {rental_num_rows} rows at row {rental_start_row}.")

# === Step 3: Total row formulas ===
total_row_values = [[
    "", "", "",                                       # A–C: leave blank
    f"=COUNTA(A{rental_start_row}:A{rental_end_row})",  # D
    f"=SUM(E{rental_start_row}:E{rental_end_row})/D{total_row_index}",  # E
    "", "",                                           # F–G
    f"=SUM(H{rental_start_row}:H{rental_end_row})",   # H
    f"=SUM(I{rental_start_row}:I{rental_end_row})",   # I
    f"=SUM(J{rental_start_row}:J{rental_end_row})"    # J
]]

spreadsheet.values_batch_update({
    "valueInputOption": "USER_ENTERED",
    "data": [{
        "range": f"'Rental Assumptions'!A{total_row_index}:J{total_row_index}",
        "values": total_row_values
    }]
})

print(f"✅ Inserted total formulas at row {total_row_index} for columns D, E, H, I, and J.")

✅ Inserted and formatted 3 rows at row 2.
✅ Inserted total formulas at row 5 for columns D, E, H, I, and J.


In [160]:
# Define ranges used in all formulas
rental_d_range = f"'Rental Assumptions'!$D${rental_start_row}:$D${rental_end_row}"
rental_e_range = f"'Rental Assumptions'!$E${rental_start_row}:$E${rental_end_row}"
rental_h_range = f"'Rental Assumptions'!$H${rental_start_row}:$H${rental_end_row}"
rental_i_range = f"'Rental Assumptions'!$I${rental_start_row}:$I${rental_end_row}"

# Build formulas row by row
avg_current_rent = []
for i in range(len(market_json)):
    row_index = market_start_row + i
    row_formulas = [
        f"=IFERROR(AVERAGEIF({rental_d_range},A{row_index},{rental_h_range}),0)",                     # Column D
        f"=IFERROR(SUMIF({rental_d_range},A{row_index},{rental_h_range})/SUMIF({rental_d_range},A{row_index},{rental_e_range}),0)",  # Column E
        f"=IFERROR(AVERAGEIF({rental_d_range},A{row_index},{rental_i_range}),0)",                     # Column F
        f"=IFERROR(SUMIF({rental_d_range},A{row_index},{rental_i_range})/SUMIF({rental_d_range},A{row_index},{rental_e_range}),0)"   # Column G
    ]
    avg_current_rent.append(row_formulas)

# Batch update columns D–G
spreadsheet.values_update(
    f"'Market Rent Assumptions'!D{market_start_row}:G{market_start_row + len(market_json) - 1}",
    params={"valueInputOption": "USER_ENTERED"},
    body={"values": avg_current_rent}
)

print("✅ Updated columns D–G in 'Market Rent Assumptions' with AVERAGEIF and SUMIF formulas.")

✅ Updated columns D–G in 'Market Rent Assumptions' with AVERAGEIF and SUMIF formulas.


In [161]:
import gspread
from gspread.utils import rowcol_to_a1

# Input data
rental_growth_json = [
    {"name": "fixed", "value": 2, "type": "rental"},
    {"name": "rent controlled", "value": 1, "type": "rental"}
]

# Setup for Assumptions sheet
assumptions_ws = spreadsheet.worksheet("Assumptions")
assumptions_start_row = 59  # insert under B58
assumptions_num_rows = len(rental_growth_json)

# Step 1: Insert blank rows into Assumptions sheet (starting at B59)
assumptions_ws.insert_rows([[]] * assumptions_num_rows, row=assumptions_start_row)

# Step 2: Populate columns B and C with data
assumptions_rows = []
for entry in rental_growth_json:
    row = [""] * 2  # B and C
    row[0] = entry["name"]           # Column B
    row[1] = entry["value"] / 100    # Column C (as decimal)
    assumptions_rows.append(row)

# Update values in B{59}:C{...}
range_str = f"'Assumptions'!B{assumptions_start_row}:C{assumptions_start_row + assumptions_num_rows - 1}"
spreadsheet.values_update(
    range_str,
    params={"valueInputOption": "USER_ENTERED"},
    body={"values": assumptions_rows}
)

# Format column C as percent
spreadsheet.batch_update({
    "requests": [
        {
            "repeatCell": {
                "range": {
                    "sheetId": assumptions_ws._properties['sheetId'],
                    "startRowIndex": assumptions_start_row - 1,
                    "endRowIndex": assumptions_start_row - 1 + assumptions_num_rows,
                    "startColumnIndex": 2,
                    "endColumnIndex": 3
                },
                "cell": {
                    "userEnteredFormat": {
                        "numberFormat": {
                            "type": "PERCENT",
                            "pattern": "0.0%"
                        }
                    }
                },
                "fields": "userEnteredFormat.numberFormat"
            }
        }
    ]
})

# Setup for Rent Roll Model sheet
rent_ws = spreadsheet.worksheet("Rent Roll Model")
rent_start_row = 3

# Step 3: Insert rows in Rent Roll Model
rent_ws.insert_rows([[]] * assumptions_num_rows, row=rent_start_row)

# Step 4: Populate Rent Roll Model with formulas referencing Assumptions!B{row}, C{row}
rent_rows = []
for i in range(assumptions_num_rows):
    assumption_row = assumptions_start_row + i
    row = [""] * 10
    row[1] = f"='Assumptions'!B{assumption_row}"  # Column B
    row[3] = f"='Assumptions'!C{assumption_row}"  # Column D
    row[9] = 1  # Column J
    rent_rows.append(row)

range_str = f"'Rent Roll Model'!A{rent_start_row}:J{rent_start_row + assumptions_num_rows - 1}"
spreadsheet.values_update(
    range_str,
    params={"valueInputOption": "USER_ENTERED"},
    body={"values": rent_rows}
)

print(f"✅ Inserted rental growth rows into 'Assumptions' at B{assumptions_start_row} and linked formulas into 'Rent Roll Model' starting at row {rent_start_row}.")

✅ Inserted rental growth rows into 'Assumptions' at B59 and linked formulas into 'Rent Roll Model' starting at row 3.


In [162]:
# Step 5: Insert formulas in NOI Walk (columns E to EF, row 16)
noi_walk_ws = spreadsheet.worksheet("NOI Walk")
noi_formula_row = 9
noi_formula_col_start = 5   # Column E
noi_formula_col_end = 140   # Column EF

# Determine MAX range in Assumptions
max_range = f"'Assumptions'!C{assumptions_start_row}:C{assumptions_start_row + assumptions_num_rows - 1}"

# Generate formulas for each column
noi_row = []
for col in range(noi_formula_col_start, noi_formula_col_end + 1):
    col_letter = rowcol_to_a1(1, col).replace("1", "")  # Convert to Excel-style letter (E, F, G, ...)
    formula = f"=(1 + (MAX({max_range}) / 12))^{col_letter}15"
    noi_row.append(formula)

# Update the full row in NOI Walk
range_str = f"'NOI Walk'!{rowcol_to_a1(noi_formula_row, noi_formula_col_start)}:{rowcol_to_a1(noi_formula_row, noi_formula_col_end)}"
spreadsheet.values_update(
    range_str,
    params={"valueInputOption": "USER_ENTERED"},
    body={"values": [noi_row]}
)

print(f"✅ Inserted growth factor formulas into 'NOI Walk' row {noi_formula_row}, columns E to EF based on MAX of Assumptions growth values.")

✅ Inserted growth factor formulas into 'NOI Walk' row 9, columns E to EF based on MAX of Assumptions growth values.


In [163]:
# import gspread
# from gspread.utils import rowcol_to_a1

# # Input data
# rental_growth_json = [
#     {"name": "fixed", "value": 2, "type": "rental"},
#     {"name": "rent controlled", "value": 1, "type": "rental"}
# ]

# # Setup
# rental_growth_ws = spreadsheet.worksheet("Rent Roll Model")
# growth_start_row = 3
# growth_num_rows = len(rental_growth_json)

# # Step 1: Insert blank rows starting at row 3
# rental_growth_ws.insert_rows([[]] * growth_num_rows, row=growth_start_row)

# # Step 2: Prepare row data for A–J
# growth_rows = []
# for entry in rental_growth_json:
#     row = [""] * 10  # Columns A–J
#     row[1] = entry["name"]                # Column A
#     row[3] = entry["value"] / 100         # Column D (as decimal for percent)
#     row[9] = 1                            # Column J
#     growth_rows.append(row)

# # Step 3: Update values in range A3:J{end}
# range_str = f"'Rent Roll Model'!A{growth_start_row}:J{growth_start_row + growth_num_rows - 1}"
# spreadsheet.values_update(
#     range_str,
#     params={"valueInputOption": "USER_ENTERED"},
#     body={"values": growth_rows}
# )

# # Step 4: Format column D as percent
# spreadsheet.batch_update({
#     "requests": [
#         {
#             "repeatCell": {
#                 "range": {
#                     "sheetId": rental_growth_ws._properties['sheetId'],
#                     "startRowIndex": growth_start_row - 1,
#                     "endRowIndex": growth_start_row - 1 + growth_num_rows,
#                     "startColumnIndex": 3,
#                     "endColumnIndex": 4
#                 },
#                 "cell": {
#                     "userEnteredFormat": {
#                         "numberFormat": {
#                             "type": "PERCENT",
#                             "pattern": "0.0%"
#                         }
#                     }
#                 },
#                 "fields": "userEnteredFormat.numberFormat"
#             }
#         }
#     ]
# })


# print(f"✅ Inserted {growth_num_rows} rental growth rows into 'Rent Roll Model' at row {growth_start_row}, with percent format in column D and 1s in column J.")

growth_start_row = 3
growth_num_rows = len(rental_growth_json)
rental_growth_ws = spreadsheet.worksheet("Rent Roll Model")

# Step 5: Add compound growth formulas (K to EK, 131 months)
start_col_index = 11  # K = 11 (1-based)
num_columns = 131
formulas = []

for i in range(growth_num_rows):
    row_num = growth_start_row + i
    row_formulas = []
    for j in range(num_columns):
        prev_col_index = start_col_index + j - 1
        prev_col_letter = rowcol_to_a1(1, prev_col_index).replace("1", "")
        formula = f"={prev_col_letter}{row_num}*(1+$D{row_num})^(1/12)"
        row_formulas.append(formula)
    formulas.append(row_formulas)

# Compute the range from K3 to EK{end}
start_col_letter = rowcol_to_a1(1, start_col_index).replace("1", "")
end_col_index = start_col_index + num_columns - 1
end_col_letter = rowcol_to_a1(1, end_col_index).replace("1", "")
range_str = f"'Rent Roll Model'!{start_col_letter}{growth_start_row}:{end_col_letter}{growth_start_row + growth_num_rows - 1}"

# Push formulas
spreadsheet.values_update(
    range_str,
    params={"valueInputOption": "USER_ENTERED"},
    body={"values": formulas}
)

print(f"✅ Inserted compound growth formulas from {start_col_letter} to {end_col_letter}, for rows {growth_start_row} to {growth_start_row + growth_num_rows - 1}.")

# Format columns K to EK as numbers with 3 decimal places
spreadsheet.batch_update({
    "requests": [
        {
            "repeatCell": {
                "range": {
                    "sheetId": rental_growth_ws._properties['sheetId'],
                    "startRowIndex": growth_start_row - 1,
                    "endRowIndex": growth_start_row - 1 + growth_num_rows,
                    "startColumnIndex": start_col_index - 1,  # K = index 10 (0-based)
                    "endColumnIndex": end_col_index,         # EK = index 141
                },
                "cell": {
                    "userEnteredFormat": {
                        "numberFormat": {
                            "type": "NUMBER",
                            "pattern": "0.000"
                        }
                    }
                },
                "fields": "userEnteredFormat.numberFormat"
            }
        }
    ]
})

print(f"✅ Formatted columns {start_col_index} to {end_col_index} (K to EK) as numbers with 3 decimal places.")

# Calculate where to start inserting in "Rent Roll Model"
start_row = 7 + len(rental_growth_json)
num_rental_rows = len(rental_assumptions_json)

# Step 1: Insert blank rows
rent_roll_ws = spreadsheet.worksheet("Rent Roll Model")
rent_roll_ws.insert_rows([[]] * num_rental_rows, row=start_row)

import gspread.utils  # for A1 notation conversion

# Step 2: Build formulas for columns A to EU (J + 131 columns)
rows_to_insert = []
for i in range(num_rental_rows):
    rental_row = 2 + i
    current_row = start_row + i

    row = [
        "",  # Column A (left blank, not written)
        f"='Rental Assumptions'!A{rental_row}",  # Column B
        f"='Rental Assumptions'!B{rental_row}",  # Column C
        f"='Rental Assumptions'!C{rental_row}",  # Column D
        f"='Rental Assumptions'!F{rental_row}",  # Column E
        f"=IF(D{current_row}=1,E{current_row}+SUM(Assumptions!$G$47:$G$48),E{current_row})",  # Column F
        f"='Rental Assumptions'!H{rental_row}",  # Column G
        f"='Rental Assumptions'!I{rental_row}",  # Column H
        ""
    ]

    # Columns J (10) to J+131 = EU (141)
    for col_idx in range(10, 142):  # 10 = J, 141 = EU
        col_letter = gspread.utils.rowcol_to_a1(1, col_idx).replace("1", "")
        formula = (
            f"=IFERROR(IF($D{current_row}=0,$G{current_row},"
            f"IF({col_letter}$8<$E{current_row},$G{current_row},"
            f"IF({col_letter}$8>=$F{current_row},$H{current_row},0)))"
            f"*INDEX({col_letter}$3:{col_letter}${3 + len(growth_rows) - 1},MATCH($C{current_row},$B$3:$B${3 + len(growth_rows) - 1},0)),0)"
        )
        row.append(formula)

    rows_to_insert.append(row)

# Determine how many columns to write (excluding A, which isn't written)
num_columns = max(len(r) for r in rows_to_insert) - 1

# Convert column index to A1 letter (B = 2, so +1)
end_col_letter = gspread.utils.rowcol_to_a1(1, num_columns + 1).replace("1", "")  # +1 to adjust for zero-based math

# Final range to write: B to computed end column
range_str = f"'Rent Roll Model'!B{start_row}:{end_col_letter}{start_row + num_rental_rows - 1}"

# Step 3: Update the sheet (excluding column A)
spreadsheet.values_update(
    range_str,
    params={"valueInputOption": "USER_ENTERED"},
    body={"values": [row[1:] for row in rows_to_insert]}  # Skip column A
)

print(f"✅ Inserted formulas into columns B–{end_col_letter} starting at row {start_row}.")


✅ Inserted compound growth formulas from K to EK, for rows 3 to 4.
✅ Formatted columns 11 to 141 (K to EK) as numbers with 3 decimal places.
✅ Inserted formulas into columns B–EK starting at row 9.


In [164]:
from gspread.utils import rowcol_to_a1

def insert_total_summary_rows(spreadsheet, sheet_name, start_row, num_rows, start_col_index=10, num_columns=132):
    ws = spreadsheet.worksheet(sheet_name)

    # === SUM ROW ===
    sum_row = start_row + num_rows
    sum_values = [""] * (start_col_index - 1)

    for col_idx in range(start_col_index, start_col_index + num_columns):
        col_letter = rowcol_to_a1(1, col_idx).replace("1", "")
        formula = f"=SUM({col_letter}{start_row}:{col_letter}{start_row + num_rows - 1})"
        sum_values.append(formula)

    start_col_letter = rowcol_to_a1(1, start_col_index).replace("1", "")
    end_col_letter = rowcol_to_a1(1, start_col_index + num_columns - 1).replace("1", "")
    sum_range = f"'{sheet_name}'!{start_col_letter}{sum_row}:{end_col_letter}{sum_row}"

    spreadsheet.values_update(
        sum_range,
        params={"valueInputOption": "USER_ENTERED"},
        body={"values": [sum_values[start_col_index - 1:]]}
    )
    print(f"✅ Inserted SUM formulas in '{sheet_name}' row {sum_row}, columns {start_col_letter} to {end_col_letter}.")

    # === INTERMEDIATE LOGIC ROWS ===
    logic_rows = []
    for r in range(3):
        row_values = [""] * (start_col_index - 1)
        for col_idx in range(start_col_index, start_col_index + num_columns):
            col_letter = rowcol_to_a1(1, col_idx).replace("1", "")
            data_range = f"{col_letter}{start_row}:{col_letter}{start_row + num_rows - 1}"

            if r == 0:
                formula = f"=COUNTIF({data_range},\">0\")"
            elif r == 1:
                formula = f"=-COUNTIF({data_range},0)"
            elif r == 2:
                formula = (
                    f"=COUNTIFS($F${start_row}:$F${start_row + num_rows - 1},{col_letter}8,"
                    f"$D${start_row}:$D${start_row + num_rows - 1},1)"
                )
            row_values.append(formula)
        logic_rows.append(row_values)

    logic_range = f"'{sheet_name}'!{start_col_letter}{sum_row+1}:{end_col_letter}{sum_row+3}"
    spreadsheet.values_update(
        logic_range,
        params={"valueInputOption": "USER_ENTERED"},
        body={"values": [r[start_col_index - 1:] for r in logic_rows]}
    )
    print(f"✅ Inserted 3 logic rows in '{sheet_name}' rows {sum_row+1} to {sum_row+3}, columns {start_col_letter} to {end_col_letter}.")

    # === REFERENCE F3/F4, H3/H4 ===
    ref_values = [
        ["=Assumptions!B51", "", "=Assumptions!G51"],
        ["=Assumptions!B53", "", "=Assumptions!G53"]
    ]
    spreadsheet.values_update(
        f"'{sheet_name}'!F3:H4",
        params={"valueInputOption": "USER_ENTERED"},
        body={"values": ref_values}
    )
    print("✅ Inserted reference formulas into F3, F4, H3, and H4.")

    # === WEIGHTED ROW ===
    weighted_row = sum_row + 4
    weighted_values = [""] * (start_col_index - 1)

    for col_idx in range(start_col_index, start_col_index + num_columns):
        col_letter = rowcol_to_a1(1, col_idx).replace("1", "")
        row_range = f"{col_letter}{start_row}:{col_letter}{start_row + num_rows - 1}"
        total_cell = f"{col_letter}{sum_row}"
        formula = (
            f"=-IF({col_letter}8<$H$3,"
            f"COUNTIFS({row_range},0)*AVERAGEIFS({row_range},{row_range},\">0\"),"
            f"{total_cell}*$H$4)"
        )
        weighted_values.append(formula)

    weighted_range = f"'{sheet_name}'!{start_col_letter}{weighted_row}:{end_col_letter}{weighted_row}"
    spreadsheet.values_update(
        weighted_range,
        params={"valueInputOption": "USER_ENTERED"},
        body={"values": [weighted_values[start_col_index - 1:]]}
    )
    print(f"✅ Inserted weighted formulas in '{sheet_name}' row {weighted_row}, columns {start_col_letter} to {end_col_letter}.")

In [165]:
insert_total_summary_rows(
    spreadsheet=spreadsheet,
    sheet_name="Rent Roll Model",
    start_row=start_row,
    num_rows=num_rental_rows
)

✅ Inserted SUM formulas in 'Rent Roll Model' row 12, columns J to EK.
✅ Inserted 3 logic rows in 'Rent Roll Model' rows 13 to 15, columns J to EK.
✅ Inserted reference formulas into F3, F4, H3, and H4.
✅ Inserted weighted formulas in 'Rent Roll Model' row 16, columns J to EK.


In [166]:
amenity_income_json = [
    {
        "id": "1",
        "name": "Parking", ## A
        "start_month": 2, ## C
        "utilization": 90, ## D
        "unit_count": 30, ## E
        "monthly_fee": 100 ## G
    },
    {
        "id": "2",
        "name": "Storage",
        "start_month": 1,
        "utilization": 20,
        "unit_count": 30,
        "monthly_fee": 100
    },
    {
        "id": "3",
        "name": "Laundry",
        "start_month": 3,
        "utilization": 40,
        "unit_count": 20,
        "monthly_fee": 200
    },
    {
        "id": "4",
        "name": "Gym/Amenity Space",
        "start_month": 4,
        "utilization": 100,
        "unit_count": 30,
        "monthly_fee": 100
    }
]

In [167]:
def insert_amenity_income(spreadsheet, sheet_name, amenity_income_json, start_row=2):
    ws = spreadsheet.worksheet(sheet_name)
    num_rows = len(amenity_income_json)

    # Step 1: Insert blank rows at row 2
    ws.insert_rows([[]] * num_rows, row=start_row)

    # Step 2: Build rows with formulas
    rows_to_insert = []
    for i, item in enumerate(amenity_income_json):
        row_num = start_row + i
        row = [
            item["name"],                   # A
            "",                             # B (unused)
            item["start_month"],           # C
            item["utilization"] / 100,       # D (as decimal)
            item["unit_count"],            # E
            f"=ROUND(E{row_num}*D{row_num},0)",      # F
            item["monthly_fee"],           # G
            f"=G{row_num}*F{row_num}",                # H
            f"=H{row_num}*12"                        # I
        ]
        rows_to_insert.append(row)

    # Step 3: Update values A2:I{end}
    range_str = f"'{sheet_name}'!A{start_row}:I{start_row + num_rows - 1}"
    spreadsheet.values_update(
        range_str,
        params={"valueInputOption": "USER_ENTERED"},
        body={"values": rows_to_insert}
    )

    print(f"✅ Inserted {num_rows} amenity income rows into '{sheet_name}' from row {start_row}.")

# Example call:
insert_amenity_income(
    spreadsheet=spreadsheet,
    sheet_name="Amenity Income",
    amenity_income_json=amenity_income_json
)

✅ Inserted 4 amenity income rows into 'Amenity Income' from row 2.


In [168]:
from gspread.utils import rowcol_to_a1

def insert_amenity_income_totals(spreadsheet, sheet_name, start_row=2, num_amenities=4):
    ws = spreadsheet.worksheet(sheet_name)
    total_row = start_row + num_amenities  # row just below the last amenity row

    h_sum_formula = f"=SUM(H{start_row}:H{total_row - 1})"
    i_sum_formula = f"=SUM(I{start_row}:I{total_row - 1})"

    # Build row with blank columns up to G (index 6), then formulas for H and I
    total_row_values = [""] * 7 + [h_sum_formula, i_sum_formula]  # Columns A–G are blank

    # Define A1 range to update H and I in total row
    range_str = f"'{sheet_name}'!A{total_row}:I{total_row}"
    spreadsheet.values_update(
        range_str,
        params={"valueInputOption": "USER_ENTERED"},
        body={"values": [total_row_values]}
    )

    print(f"✅ Inserted SUM formulas into row {total_row} for columns H and I on '{sheet_name}'.")

# Example call:
insert_amenity_income_totals(
    spreadsheet=spreadsheet,
    sheet_name="Amenity Income",
    start_row=2,
    num_amenities=len(amenity_income_json)
)

✅ Inserted SUM formulas into row 6 for columns H and I on 'Amenity Income'.


In [169]:
from gspread.utils import rowcol_to_a1

def insert_amenity_income_formulas(spreadsheet, amenity_json, noi_sheet="NOI Walk", amenity_sheet="Amenity Income", start_row=25, start_col=5, num_months=131):
    num_amenities = len(amenity_json)
    ws = spreadsheet.worksheet(noi_sheet)

    # ✅ Step 1: Insert empty rows
    ws.insert_rows([[]] * num_amenities, row=start_row)

    # ✅ Step 2: Build full row values with:
    # - Column B: =‘Amenity Income’!A{row}
    # - Columns E →: Monthly formulas
    all_rows = []

    for i in range(num_amenities):
        amenity_row = 2 + i  # Row on 'Amenity Income'
        noi_row = start_row + i  # Corresponding row on 'NOI Walk'

        row = [""]  # Column A (left blank)
        row.append(f"='{amenity_sheet}'!A{amenity_row}")  # Column B
        row += ["", "", ""]  # Columns C, D, unused E slot up to column E (start_col=5)

        for j in range(num_months):
            col_index = start_col + j
            col_letter = rowcol_to_a1(1, col_index).replace("1", "")
            formula = (
                f"=IF({col_letter}$15<'{amenity_sheet}'!$C{amenity_row},"
                f"0,"
                f"'{amenity_sheet}'!G{amenity_row}*{col_letter}$16*{col_letter}$10*'{amenity_sheet}'!D{amenity_row})"
            )
            row.append(formula)

        all_rows.append(row)

    # ✅ Step 3: Define target range and update
    total_cols = len(all_rows[0])
    end_col_letter = rowcol_to_a1(1, total_cols).replace("1", "")
    end_row = start_row + num_amenities - 1

    range_str = f"'{noi_sheet}'!A{start_row}:{end_col_letter}{end_row}"

    spreadsheet.values_update(
        range_str,
        params={"valueInputOption": "USER_ENTERED"},
        body={"values": all_rows}
    )

    print(f"✅ Inserted {num_amenities} amenity rows into '{noi_sheet}' rows {start_row}–{end_row}, including column B references and month formulas from {rowcol_to_a1(1, start_col).replace('1','')} to {end_col_letter}.")

In [170]:

# Example call:
insert_amenity_income_formulas(
    spreadsheet=spreadsheet,
    amenity_json=amenity_income_json,
    noi_sheet="NOI Walk",
    amenity_sheet="Amenity Income",
    start_row=25,
    start_col=5,  # Column E
    num_months=131
)

✅ Inserted 4 amenity rows into 'NOI Walk' rows 25–28, including column B references and month formulas from E to EF.


In [171]:
def insert_noi_summary_rows(spreadsheet, num_rows, walk_start_row=25, noi_start_row=10, noi_sheet="NOI", walk_sheet="NOI Walk"):
    noi_ws = spreadsheet.worksheet(noi_sheet)

    # Step 1: Insert blank rows
    noi_ws.insert_rows([[]] * num_rows, row=noi_start_row)

    # Step 2: Build new rows
    all_rows = []
    for i in range(num_rows):
        walk_row = walk_start_row + i
        noi_row = noi_start_row + i

        row = [""] * 2  # Column A (blank)
        row.append(f"='{walk_sheet}'!B{walk_row}")  # Column B

        row += ["", "", ""]  # Columns C, D, E, F

        # Columns G–K
        for col_letter in ["G", "H", "I", "J", "K"]:
            formula = (
                f"=SUMIFS('{walk_sheet}'!$E{walk_row}:$CW{walk_row},"
                f"'{walk_sheet}'!$E$14:$CW$14,"
                f"{noi_sheet}!{col_letter}$4)"
            )
            row.append(formula)

        all_rows.append(row)

    # Step 3: Define range and update
    end_col_letter = rowcol_to_a1(1, len(all_rows[0])).replace("1", "")
    end_row = noi_start_row + num_rows - 1
    range_str = f"'{noi_sheet}'!A{noi_start_row}:{end_col_letter}{end_row}"

    spreadsheet.values_update(
        range_str,
        params={"valueInputOption": "USER_ENTERED"},
        body={"values": all_rows}
    )

    print(f"✅ Inserted {num_rows} summary rows into '{noi_sheet}' starting at row {noi_start_row}, with B linking to '{walk_sheet}' and SUMIFS in G–K.")
insert_noi_summary_rows(spreadsheet, num_rows=len(amenity_income_json))

✅ Inserted 4 summary rows into 'NOI' starting at row 10, with B linking to 'NOI Walk' and SUMIFS in G–K.


In [172]:
expenses_json = [
    {
        "id": "1",
        "name": "Property Taxes", ## A
        "factor": 4000, ## F
        "broker": 300000 ## J
    },
    {
        "id": "2",
        "name": "Insurance",
        "factor": 910,
        "broker": 60000
    },
    {
        "id": "3",
        "name": "Water / Sewer",
        "factor": 600,
        "broker": 0
    },
    {
        "id": "4",
        "name": "Repairs & Maintenance",
        "factor": 600,
        "broker": 0
    },
    {
        "id": "5",
        "name": "Bank fees",
        "factor": 10,
        "broker": 0
    },
    {
        "id": "6",
        "name": "Trash",
        "factor": 20,
        "broker": 0
    },
    {
        "id": "7",
        "name": "Cleaning",
        "factor": 20,
        "broker": 0
    },
    {
        "id": "8",
        "name": "Security",
        "factor": 100,
        "broker": 0
    },
    {
        "id": "9",
        "name": "Internet",
        "factor": 97,
        "broker": 0
    },
    {
        "id": "10",
        "name": "Pest Control",
        "factor": 100,
        "broker": 0
    },
    {
        "id": "11",
        "name": "Super (Payroll)",
        "factor": 100,
        "broker": 0
    },
    {
        "id": "12",
        "name": "Heat (Gas)",
        "factor": 100,
        "broker": 0
    },
    {
        "id": "13",
        "name": "Common Area Electric",
        "factor": 197,
        "broker": 0
    },
    {
        "id": "14",
        "name": "Landscaping / Snow Removal",
        "factor": 200,
        "broker": 0
    },
    {
        "id": "15",
        "name": "Property Management",
        "factor": 100,
        "broker": 0
    },
    {
        "id": "16",
        "name": "Accounting / Legal",
        "factor": 200,
        "broker": 0
    }
]

In [173]:
def insert_operating_expenses(spreadsheet, expenses_json, rental_assumptions_json, amenity_income_json, sheet_name="Operating Expenses", start_row=2):
    ws = spreadsheet.worksheet(sheet_name)
    num_rows = len(expenses_json)
    ws.insert_rows([[]] * num_rows, row=start_row)

    rental_row = len(rental_assumptions_json) + 2
    amenity_row = len(amenity_income_json) + 2

    rows_to_insert = []

    for i, expense in enumerate(expenses_json):
        row_num = start_row + i
        row = [""] * 10  # Columns A–J
        row[0] = expense["name"]       # Column A
        row[5] = expense["factor"]     # Column F
        row[9] = expense["broker"]     # Column J

        name = expense["name"]

        # Column G logic
        col_g_formula = ""
        if name in [
            "Property Taxes", "Insurance", "Water / Sewer", "Repairs & Maintenance",
            "Bank fees", "Cleaning", "Pest Control", "Super (Payroll)", "Heat (Gas)"
        ]:
            col_g_formula = f"='Rental Assumptions'!$D${rental_row}"
        elif name == "Common Area Electric":
            col_g_formula = "=Assumptions!$G$24-Assumptions!G25"
        elif name == "Property Management":
            col_g_formula = (
                f"=('Rental Assumptions'!$J${rental_row}+'Amenity Income'!$I${amenity_row}+'Retail Income'!I2)"
                f"*(1-Assumptions!G53-Assumptions!K51)"
            )
        row[6] = col_g_formula  # Column G

        # Column I logic (depends on G)
        col_i_formula = f"=F{row_num}*G{row_num}" if col_g_formula else f"=F{row_num}"
        row[8] = col_i_formula  # Column I

        # Column H logic
        row[7] = f"=I{row_num}/12"  # Column H

        rows_to_insert.append(row)

    end_row = start_row + num_rows - 1
    range_str = f"'{sheet_name}'!A{start_row}:J{end_row}"
    spreadsheet.values_update(
        range_str,
        params={"valueInputOption": "USER_ENTERED"},
        body={"values": rows_to_insert}
    )

    print(f"✅ Inserted {num_rows} operating expenses into '{sheet_name}' with correct formulas for columns G, H, I, and J.")

In [174]:
insert_operating_expenses(
    spreadsheet=spreadsheet,
    expenses_json=expenses_json,  # Predefined list
    rental_assumptions_json=rental_assumptions_json,  # Predefined list
    amenity_income_json=amenity_income_json
)

✅ Inserted 16 operating expenses into 'Operating Expenses' with correct formulas for columns G, H, I, and J.


In [175]:
def insert_operating_expenses_sums(spreadsheet, expenses_json, sheet_name="Operating Expenses", start_row=2):
    num_expenses = len(expenses_json)
    sum_row = start_row + num_expenses  # Row after last expense

    h_sum_formula = f"=SUM(H{start_row}:H{sum_row - 1})"
    i_sum_formula = f"=SUM(I{start_row}:I{sum_row - 1})"
    j_sum_formula = f"=SUM(J{start_row}:J{sum_row - 1})"

    # Update range H{sum_row}:J{sum_row}
    range_str = f"'{sheet_name}'!H{sum_row}:J{sum_row}"
    spreadsheet.values_update(
        range_str,
        params={"valueInputOption": "USER_ENTERED"},
        body={"values": [[h_sum_formula, i_sum_formula, j_sum_formula]]}
    )

    print(f"✅ Inserted SUM formulas into '{sheet_name}'!H{sum_row}, I{sum_row}, and J{sum_row}.")
    
insert_operating_expenses_sums(
    spreadsheet=spreadsheet,
    expenses_json=expenses_json,
    sheet_name="Operating Expenses",  # Optional if you're using a different sheet name
    start_row=2  # Assuming expenses start in row 2
)

✅ Inserted SUM formulas into 'Operating Expenses'!H18, I18, and J18.


In [176]:
from gspread.utils import rowcol_to_a1

def insert_operating_expense_formulas_to_noi_walk(spreadsheet, expenses_json, amenity_income_json, noi_sheet="NOI Walk", op_exp_sheet="Operating Expenses", start_base_row=34, start_col=5, num_months=132):
    start_row = start_base_row + len(amenity_income_json)
    num_expenses = len(expenses_json)
    end_row = start_row + num_expenses - 1
    end_col_index = start_col + num_months - 1

    ws = spreadsheet.worksheet(noi_sheet)

    # Step 1: Insert blank rows using batchUpdate
    requests = [
        {
            "insertDimension": {
                "range": {
                    "sheetId": ws._properties["sheetId"],
                    "dimension": "ROWS",
                    "startIndex": start_row - 1,  # 0-indexed
                    "endIndex": start_row - 1 + num_expenses
                },
                "inheritFromBefore": False
            }
        }
    ]
    spreadsheet.batch_update({"requests": requests})
    print(f"✅ Inserted {num_expenses} blank rows at {noi_sheet}!{start_row}")

    # Step 2: Column B references from 'Operating Expenses'
    b_values = [[f"='{op_exp_sheet}'!A{2 + i}"] for i in range(num_expenses)]
    b_range = f"'{noi_sheet}'!B{start_row}:B{end_row}"

    # Step 3: Monthly formulas E to EQ
    formula_rows = []
    for i in range(num_expenses):
        op_row = 2 + i
        formula_row = []
        for j in range(num_months):
            col_index = start_col + j
            col_letter = rowcol_to_a1(1, col_index).replace("1", "")
            formula = f"='{op_exp_sheet}'!$H{op_row}*{col_letter}$11"
            formula_row.append(formula)
        formula_rows.append(formula_row)

    start_col_letter = rowcol_to_a1(1, start_col).replace("1", "")
    end_col_letter = rowcol_to_a1(1, end_col_index).replace("1", "")
    e_range = f"'{noi_sheet}'!{start_col_letter}{start_row}:{end_col_letter}{end_row}"

    # Step 4: Push both B column and E-EQ formulas using batch
    spreadsheet.values_batch_update(
        {
            "valueInputOption": "USER_ENTERED",
            "data": [
                {"range": b_range, "values": b_values},
                {"range": e_range, "values": formula_rows}
            ]
        }
    )

    print(f"✅ Inserted formulas into '{noi_sheet}' columns B and {start_col_letter}–{end_col_letter} for rows {start_row}–{end_row}")

In [177]:
insert_operating_expense_formulas_to_noi_walk(
    spreadsheet=spreadsheet,
    expenses_json=expenses_json,
    amenity_income_json=amenity_income_json,
    noi_sheet="NOI Walk",
    op_exp_sheet="Operating Expenses",
    start_base_row=30,
    start_col=5,  # Column E
    num_months=132
)

✅ Inserted 16 blank rows at NOI Walk!34
✅ Inserted formulas into 'NOI Walk' columns B and E–EF for rows 34–49


In [178]:
from gspread.utils import rowcol_to_a1

def insert_expense_sum_row_to_noi_walk(spreadsheet, amenity_income_json, expenses_json, sheet_name="NOI Walk", start_col=5, num_months=132, row_offset=30):
    start_sum_row = row_offset + len(amenity_income_json) + len(expenses_json)
    row_start = row_offset + len(amenity_income_json)  # e.g., 30 + len(amenities)
    row_end = row_start + len(expenses_json) - 1

    sum_row = []

    for j in range(num_months):
        col_index = start_col + j
        col_letter = rowcol_to_a1(1, col_index).replace("1", "")
        formula = f"=SUM({col_letter}{row_start}:{col_letter}{row_end})"
        sum_row.append(formula)

    start_col_letter = rowcol_to_a1(1, start_col).replace("1", "")
    end_col_letter = rowcol_to_a1(1, start_col + num_months - 1).replace("1", "")
    range_str = f"'{sheet_name}'!{start_col_letter}{start_sum_row}:{end_col_letter}{start_sum_row}"

    spreadsheet.values_update(
        range_str,
        params={"valueInputOption": "USER_ENTERED"},
        body={"values": [sum_row]}
    )

    print(f"✅ Inserted SUM formulas into '{sheet_name}' row {start_sum_row}, columns {start_col_letter} to {end_col_letter}")

In [179]:
insert_expense_sum_row_to_noi_walk(
    spreadsheet=spreadsheet,
    amenity_income_json=amenity_income_json,
    expenses_json=expenses_json
)

✅ Inserted SUM formulas into 'NOI Walk' row 50, columns E to EF


In [180]:
def insert_noi_expense_rows(spreadsheet, expenses_json, amenity_income_json, walk_start_row=34, noi_base_row=14, noi_sheet="NOI", walk_sheet="NOI Walk"):
    noi_ws = spreadsheet.worksheet(noi_sheet)

    num_rows = len(expenses_json)
    noi_start_row = noi_base_row + len(amenity_income_json)

    # Step 1: Insert blank rows
    noi_ws.insert_rows([[]] * num_rows, row=noi_start_row)

    # Step 2: Build row data
    all_rows = []
    for i in range(num_rows):
        walk_row = walk_start_row + i
        row = [""] * 2  # Columns A–B
        row.append(f"='{walk_sheet}'!B{walk_row}")  # Column C
        row += ["", "", ""]  # Columns D–F

        # Columns G–K
        for col_letter in ["G", "H", "I", "J", "K"]:
            formula = (
                f"=SUMIFS('{walk_sheet}'!$E{walk_row}:$CW{walk_row},"
                f"'{walk_sheet}'!$E$14:$CW$14,"
                f"{noi_sheet}!{col_letter}$4)"
            )
            row.append(formula)

        all_rows.append(row)

    # Step 3: Upload to sheet
    end_col_letter = rowcol_to_a1(1, len(all_rows[0])).replace("1", "")
    end_row = noi_start_row + num_rows - 1
    range_str = f"'{noi_sheet}'!A{noi_start_row}:{end_col_letter}{end_row}"

    spreadsheet.values_update(
        range_str,
        params={"valueInputOption": "USER_ENTERED"},
        body={"values": all_rows}
    )

    print(f"✅ Inserted {num_rows} NOI expense summary rows into '{noi_sheet}' starting at row {noi_start_row}, with C referencing '{walk_sheet}' and SUMIFS in G–K.")
insert_noi_expense_rows(spreadsheet, expenses_json, amenity_income_json)

✅ Inserted 16 NOI expense summary rows into 'NOI' starting at row 18, with C referencing 'NOI Walk' and SUMIFS in G–K.


In [181]:
from gspread.utils import rowcol_to_a1

def insert_ntm(
    spreadsheet,
    amenity_income_json,
    expenses_json,
    sheet_name="NOI Walk",
    target_start_col=5,   # Column E
    target_num_cols=132,
    formula_start_col=6,  # Column F
    formula_range_width=11,  # F:Q = 11 columns
    row_offset=31
):
    row_1 = row_offset + len(amenity_income_json) + len(expenses_json) + 1
    row_2 = row_offset + len(amenity_income_json) + len(expenses_json) - 2
    target_row = row_offset + len(amenity_income_json) + len(expenses_json) + 2  # <-- 1 above prior

    values = []
    for i in range(target_num_cols):
        start_idx = formula_start_col + i
        end_idx = start_idx + formula_range_width
        start_letter = rowcol_to_a1(1, start_idx).replace("1", "")
        end_letter = rowcol_to_a1(1, end_idx).replace("1", "")
        formula = f"=SUM({start_letter}{row_1}:{end_letter}{row_1})"
        values.append(formula)

    # Define the range to update
    output_start_letter = rowcol_to_a1(1, target_start_col).replace("1", "")
    output_end_letter = rowcol_to_a1(1, target_start_col + target_num_cols - 1).replace("1", "")
    range_str = f"'{sheet_name}'!{output_start_letter}{target_row}:{output_end_letter}{target_row}"

    print(f"➡️ Inserting into row: {target_row}")
    print(f"➡️ First formula: {values[0]}")
    print(f"➡️ Last formula: {values[-1]}")
    print(f"➡️ Full range: {range_str}")

    spreadsheet.values_update(
        range_str,
        params={"valueInputOption": "USER_ENTERED"},
        body={"values": [values]}
    )

    print(f"✅ Inserted incrementing NTM formulas into '{sheet_name}' row {target_row}, columns {output_start_letter} to {output_end_letter}")

In [182]:
insert_ntm(
    spreadsheet=spreadsheet,
    amenity_income_json=amenity_income_json,
    expenses_json=expenses_json
)

➡️ Inserting into row: 53
➡️ First formula: =SUM(F52:Q52)
➡️ Last formula: =SUM(EG52:ER52)
➡️ Full range: 'NOI Walk'!E53:EF53
✅ Inserted incrementing NTM formulas into 'NOI Walk' row 53, columns E to EF


In [183]:
def update_closing_costs_cell(spreadsheet, sheet_name="Closing Costs"):
    cell = "D13"
    formula = "=+'Operating Expenses'!I2/4"
    range_str = f"'{sheet_name}'!{cell}"

    spreadsheet.values_update(
        range_str,
        params={"valueInputOption": "USER_ENTERED"},
        body={"values": [[formula]]}
    )

    print(f"✅ Updated {sheet_name}!{cell} to '{formula}'")

update_closing_costs_cell(spreadsheet)

✅ Updated Closing Costs!D13 to '=+'Operating Expenses'!I2/4'


In [184]:
# import gspread.utils  # for A1 notation conversion if needed

# # Step 2: Build formulas for columns A to EK
# rows_to_insert = []
# for i in range(num_rental_rows):
#     rental_row = 2 + i
#     current_row = start_row + i

#     row = [
#         "",  # Column A (left blank, won't be inserted)
#         f"='Rental Assumptions'!A{rental_row}",  # B
#         f"='Rental Assumptions'!B{rental_row}",  # C
#         f"='Rental Assumptions'!C{rental_row}",  # D
#         f"='Rental Assumptions'!F{rental_row}",  # E
#         f"=IF(D{current_row}=1,E{current_row}+SUM(Assumptions!$G$47:$G$48),E{current_row})",  # F
#         f"='Rental Assumptions'!H{rental_row}",  # G
#         f"='Rental Assumptions'!I{rental_row}",  # H
#         # ... add more columns as needed up to column EK
#         # f"=..." for column I
#         # ...
#     ]
#     rows_to_insert.append(row)

# # Confirm how many columns you're inserting
# num_columns = max(len(r) for r in rows_to_insert)

# # Convert column index to letter (1-indexed: A=1, B=2, ..., EK=137)
# end_col_letter = gspread.utils.rowcol_to_a1(1, num_columns + 1).replace("1", "")  # +1 since col A is skipped

# # Step 3: Write values into range B:EK (or whatever the end column is)
# range_str = f"'Rent Roll Model'!B{start_row}:{end_col_letter}{start_row + num_rental_rows - 1}"

# spreadsheet.values_update(
#     range_str,
#     params={"valueInputOption": "USER_ENTERED"},
#     body={"values": [row[1:] for row in rows_to_insert]}  # Exclude column A from insert
# )

# print(f"✅ Inserted rows with references and conditional logic into columns B–{end_col_letter} starting at row {start_row}.")

In [185]:

# # Define rows and ranges
# start_row = 2
# num_rows = len(example_json)
# rows = []

# rows = []
# for i, item in enumerate(example_json):
#     row_num = start_row + i

#     pf_rent_formula = (
#         f"=IFERROR("
#         f"IF(OR(C{row_num}=1,C{row_num}=2),"
#         f"INDEX('Market Rent Assumptions'!$C${market_start_row}:$C${market_end_row}, "
#         f"MATCH(D{row_num}, 'Market Rent Assumptions'!$A${market_start_row}:$A${market_end_row}, 0)),"
#         f"H{row_num}),"
#         f"H{row_num})"
#     )

#     annual_pf_rent_formula = f"=I{row_num}*12"

#     row = [
#         i + 1,                                        # A: Units
#         item["rent_type"],                           # B: Rent Type
#         item["vacate_flag"],                         # C: Vacate Flag
#         item["layout"],                              # D: Layout
#         item["square_feet"],                         # E: Sq. Ft.
#         item["vacate_month"],                        # F: Vacated Month
#         f"=EOMONTH(Assumptions!$F$2,F{row_num})",    # G: Vacated
#         item["current_rent"],                        # H: Current Rent
#         pf_rent_formula,                             # I: Pro Forma Rent
#         annual_pf_rent_formula                       # J: Annual PF Rent 
#     ]
#     rows.append(row)

# # Batch update with both value insert and formatting
# spreadsheet.batch_update({
#     "requests": [
#         # Insert blank rows to push content down
#         {
#             "insertDimension": {
#                 "range": {
#                     "sheetId": worksheet._properties['sheetId'],
#                     "dimension": "ROWS",
#                     "startIndex": start_row - 1,
#                     "endIndex": start_row - 1 + num_rows
#                 },
#                 "inheritFromBefore": True
#             }
#         },
#         # Format "Vacated" column as Date
#         {
#             "repeatCell": {
#                 "range": {
#                     "sheetId": worksheet._properties['sheetId'],
#                     "startRowIndex": start_row - 1,
#                     "endRowIndex": start_row - 1 + num_rows,
#                     "startColumnIndex": 6,
#                     "endColumnIndex": 7
#                 },
#                 "cell": {
#                     "userEnteredFormat": {
#                         "numberFormat": {
#                             "type": "DATE",
#                             "pattern": "mm/dd/yyyy"
#                         }
#                     }
#                 },
#                 "fields": "userEnteredFormat.numberFormat"
#             }
#         }
#     ]
# })

# # Perform value update with formulas using USER_ENTERED mode
# spreadsheet.values_update(
#     f"'Rental Assumptions'!A{start_row}:J{start_row + num_rows - 1}",
#     params={"valueInputOption": "USER_ENTERED"},
#     body={"values": rows}
# )

# print(f"✅ Inserted and formatted {num_rows} rows at row {start_row}.")

# # Total row is the row just after the inserted block
# total_row_index = start_row + num_rows

# # Define all formulas
# total_d = f"=COUNTA(A{start_row}:A{start_row + num_rows - 1})"
# total_e = f"=SUM(E{start_row}:E{start_row + num_rows - 1})/D{total_row_index}"
# total_h = f"=SUM(H{start_row}:H{start_row + num_rows - 1})"
# total_i = f"=SUM(I{start_row}:I{start_row + num_rows - 1})"
# total_j = f"=SUM(J{start_row}:J{start_row + num_rows - 1})"

# # Build all values in one row (match columns D–J)
# total_row_values = [["", "", "", total_d, total_e, "", "", total_h, total_i, total_j]]

# # Write all formulas to columns D–J on the total row
# spreadsheet.values_batch_update(
#     body={
#         "valueInputOption": "USER_ENTERED",
#         "data": [
#             {
#                 "range": f"'Rental Assumptions'!A{total_row_index}:J{total_row_index}",
#                 "values": total_row_values
#             }
#         ]
#     }
# )

# print(f"✅ Inserted total formulas at row {total_row_index} for columns D, E, H, I, and J.")